# Ejemplo 008

**Problema:** Queremos detectar cambios en el reporte de Radios aproximados de sectores de Ingenier@, y leer todos los registros que se modificaron. 


#### Importamos paquetes necesarios

In [18]:
# Importación de paquetes requeridos
import os
import io
import requests
import pandas as pd
from datetime import datetime
from IPython.display import display, Markdown, Latex

import json

#### Funciones auxiliares

In [3]:
# Función auxiliar para visualizar los datos
def pp(obj):
    print(json.dumps(obj, indent=2))

#### Parámetros del script

In [49]:
# Base URL de la api de Ingenier@
iga_api_base_url = "http://ing-acc-movil01.personal.corp/ingenier@/symfony/public/index.php/api"

# Última sincronización: última vez que se ejecutó este script. 
# Debemos obtener todos los registros que se modificaron o agregaron posteriormente a este datetime.
last_sync_datetime = "2024-06-11 18:15:11.000000"


#### Descarga de reportes

- Debemos descargar el reporte de Radios aproximados de sectores. 
- Los sitios que nos interesan son los on air (OA), es decir que tienen al menos una celda operativa.
- El endpoint usado es /public/report/generic-report.
- Los datos devueltos están en formato binario, y pueden ser leídos directamente por Pandas como un archivo .xlsx, y convertidos a un objeto DataFrame

In [45]:
# Parámetros en el payload de la request con que llamamos al endpoint de IGA:
payload = {
    # Nombre del reporte
    "reportName": "datos_radio_sector",
    # Sitios seleccionados: ninguno
    "siteNames": [],
    # Parámetros de filtrado: todos los sitios en estado operativo (OA).
    # Son los sitios que tienen al menos una celda operativa.
    "filteringParams": {"SiteVisualizationData.SiteStatus": ["OA"]},
    # Seleccionar por nombres de sitio: no
    "selectBySiteNames": False,
    # Seleccionar por parámetros de filtrado: sí
    "selectByFilteringParams": True
}

x = requests.post(iga_api_base_url + '/public/report/generic-report', json=payload)
with io.BytesIO(x.content) as fh:
    df_radio_sector_raw = pd.read_excel(fh, "Radios aproximados de sectores")

display(df_radio_sector_raw)

,Código Sitio,EMG,Estado Sitio,Tipo Estación,Sector,Tecnol./Banda,Estado Sector,Radio Aprox. Sector [m],Última Modif.
0,A01-E133,XRCESE,OA,EasyM,XRCESEL1G,LTE FDD_AWS_2225,Operativo,618.326588,2024-06-11 18:15:10
1,A01-E133,XRCESE,OA,EasyM,XRCESEL1H,LTE FDD_AWS_2225,Operativo,573.343038,2024-06-11 18:15:10
2,A01-E133,XRCESE,OA,EasyM,XRCESEL1I,LTE FDD_AWS_2225,Operativo,713.040137,2024-06-11 18:15:10
3,A01-E133,XRCESE,OA,EasyM,XRCESEM1G,LTE FDD_1900MHz_975,No Operativo,618.326588,2024-06-11 18:15:10
4,A01-E133,XRCESE,OA,EasyM,XRCESEM1H,LTE FDD_1900MHz_975,No Operativo,573.343038,2024-06-11 18:15:10
...,...,...,...,...,...,...,...,...,...
166498,SPU990,SBBORI,OA,Macro,SBBORIU12,WCDMA1900_9788,No Operativo,4492.913208,2024-06-11 18:15:10
166499,SPU990,SBBORI,OA,Macro,SBBORIU13,WCDMA1900_9788,No Operativo,2678.886356,2024-06-11 18:15:10
166500,SPU990,SBBORI,OA,Macro,SBBORIV21,WCDMA850_4358,No Operativo,3991.228227,2024-06-11 18:15:10
166501,SPU990,SBBORI,OA,Macro,SBBORIV22,WCDMA850_4358,No Operativo,3991.228227,2024-06-11 18:15:10


#### Procesamiento de los datos

- Filtramos las celdas y nos quedamos únicamente con las que estén en estado operativo
- Nos quedamos además solo con las celdas que fueron modificadas luego de la última sincronización

In [50]:
df_radio_sector = (
    df_radio_sector_raw[
        (df_radio_sector_raw['Estado Sector'] == 'Operativo')
        & (df_radio_sector_raw['Última Modif.'] > last_sync_datetime)
    ]
)
display(df_radio_sector)

,Código Sitio,EMG,Estado Sitio,Tipo Estación,Sector,Tecnol./Banda,Estado Sector,Radio Aprox. Sector [m],Última Modif.
28946,A07-S053,ZMESCA,OA,Macro,ZMESCAM12,LTE FDD_1900MHz_976,Operativo,5213.805969,2024-06-11 18:16:10
28948,A07-S053,ZMESCA,OA,Macro,ZMESCAN11,LTE FDD_700MHz_9360,Operativo,7268.462532,2024-06-11 18:16:10
28950,A07-S053,ZMESCA,OA,Macro,ZMESCAN13,LTE FDD_700MHz_9360,Operativo,7892.614459,2024-06-11 18:16:10
28952,A07-S053,ZMESCA,OA,Macro,ZMESCAO12,LTE FDD_850MHz_2476,Operativo,8353.261302,2024-06-11 18:16:10
28954,A07-S053,ZMESCA,OA,Macro,ZMESCAV11,WCDMA850_4358,Operativo,8353.261302,2024-06-11 18:16:10
28956,A07-S053,ZMESCA,OA,Macro,ZMESCAV13,WCDMA850_4358,Operativo,8353.261302,2024-06-11 18:16:10
28964,A07-S055,EBOVRI,OA,Macro,EBOVRIL12,LTE FDD_AWS_2225,Operativo,5195.496490,2024-06-11 18:16:10


In [51]:
# Podemos convertirlo a formato json
print(df_radio_sector.to_json(force_ascii=False, indent=2, orient='records'))

[
  {
    "Código Sitio":"A07-S053",
    "EMG":"ZMESCA",
    "Estado Sitio":"OA",
    "Tipo Estación":"Macro",
    "Sector":"ZMESCAM12",
    "Tecnol.\/Banda":"LTE FDD_1900MHz_976",
    "Estado Sector":"Operativo",
    "Radio Aprox. Sector [m]":5213.805968737,
    "Última Modif.":1718129770000
  },
  {
    "Código Sitio":"A07-S053",
    "EMG":"ZMESCA",
    "Estado Sitio":"OA",
    "Tipo Estación":"Macro",
    "Sector":"ZMESCAN11",
    "Tecnol.\/Banda":"LTE FDD_700MHz_9360",
    "Estado Sector":"Operativo",
    "Radio Aprox. Sector [m]":7268.462531903,
    "Última Modif.":1718129770000
  },
  {
    "Código Sitio":"A07-S053",
    "EMG":"ZMESCA",
    "Estado Sitio":"OA",
    "Tipo Estación":"Macro",
    "Sector":"ZMESCAN13",
    "Tecnol.\/Banda":"LTE FDD_700MHz_9360",
    "Estado Sector":"Operativo",
    "Radio Aprox. Sector [m]":7892.614458843,
    "Última Modif.":1718129770000
  },
  {
    "Código Sitio":"A07-S053",
    "EMG":"ZMESCA",
    "Estado Sitio":"OA",
    "Tipo Estación":"Macro"

In [52]:
# Luego de impactar los datos deberíamos actualizar el valor de last_sync_datetime con fecha 
# y hora actual y persistirlo antes de la próxima ejecución del script.

last_sync_datetime = datetime.now()
print(last_sync_datetime)


2024-06-12 10:15:47.585396
